In [24]:
act MAPPING

(MAPPING) 

: 1

# define the user information

In [49]:
mode="long"
ref_dir="test"
reads="/g2_4t_ssd/junsheng/Test_nano_illu_hybrid/OPERA_MS_GIS20_ERX3228429/original/ONT/ERR3200811.fastq"

(MAPPING) (MAPPING) (MAPPING) 

: 1

# get the program

In [9]:
p_short=$(which bwa)
p_long=$(which minimap2)
p_sam=$(which samtools)


if [ $mode == "long" ];then

    if [ -z $p_long ];then
        echo "No minimap2."
        exit 1
    else
        echo "The path of minimap2 is $p_long"
    fi

elif [ $mode == "short" ];then

    if [ -z $p_short ];then
        echo "No bwa."
    else
        echo "The path of minimap2 is $p_short"
    fi
fi


if [ -z $p_sam ];then
    echo "No samtools."
    exit 1
else
    echo "The path of samtools is $p_sam"
fi

The path of minimap2 is /home/junsheng/BIN/ont-guppy/bin/minimap2
The path of samtools is /usr/bin/samtools


# use minimap2 to index the reference

In [13]:
find $ref_dir -type f | parallel "$p_long -d {}.mmi {} "

[M::mm_idx_gen::0.050*1.03] collected minimizers
[M::mm_idx_gen::0.074*1.04] sorted minimizers
[M::main::0.110*0.89] loaded/built the index for 2 target sequence(s)
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 2
[M::mm_idx_stat::0.123*0.85] distinct minimizers: 324721 (98.27% are singletons); average occurrences: 1.027; average spacing: 5.366; total length: 1790058
[M::main] Version: 2.22-r1101
[M::main] CMD: /home/junsheng/BIN/ont-guppy/bin/minimap2 -d references/Jonquetella_anthropi.fna.gz.mmi references/Jonquetella_anthropi.fna.gz
[M::main] Real time: 0.138 sec; CPU: 0.120 sec; Peak RSS: 0.020 GB
[M::mm_idx_gen::0.080*1.02] collected minimizers
[M::mm_idx_gen::0.112*1.01] sorted minimizers
[M::main::0.137*1.01] loaded/built the index for 38 target sequence(s)
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 38
[M::mm_idx_stat::0.147*1.01] distinct minimizers: 395868 (98.35% are singletons); average occurrences: 1.040; average spacing: 5.336; total length: 219

# mapping

In [11]:
threads=$(cat /proc/cpuinfo | grep "processor" |wc -l)

In [18]:
find $ref_dir -type f -name "*.mmi"| parallel -j 1 "$p_long -ax map-ont -t $threads {} $reads > {}.sam "

[M::main::0.015*1.03] loaded/built the index for 42 target sequence(s)
[M::mm_mapopt_update::0.021*1.02] mid_occ = 10
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 42
[M::mm_idx_stat::0.025*1.02] distinct minimizers: 335233 (97.78% are singletons); average occurrences: 1.029; average spacing: 5.328; total length: 1838801
[M::worker_pipeline::45.215*0.91] mapped 110709 sequences
[M::worker_pipeline::84.298*0.91] mapped 149017 sequences
[M::worker_pipeline::120.531*0.91] mapped 105114 sequences
[M::worker_pipeline::161.873*0.92] mapped 85237 sequences
[M::worker_pipeline::189.846*0.85] mapped 82586 sequences
[M::main] Version: 2.22-r1101
[M::main] CMD: /home/junsheng/BIN/ont-guppy/bin/minimap2 -ax map-ont -t 16 references/Finegoldia_magna.fna.gz.mmi /g2_4t_ssd/junsheng/Test_nano_illu_hybrid/OPERA_MS_GIS20_ERX3228429/original/ONT/ERR3200811.fastq
[M::main] Real time: 190.435 sec; CPU: 161.676 sec; Peak RSS: 2.347 GB
[M::main::0.018*1.02] loaded/built the index for 2 target se

# to bam, sort

In [54]:
find $ref_dir -type f -name "*.sam"|parallel -j 1 "samtools view -uS -@$threads {}|samtools sort -@$threads -o {}.sort.bam -"

[bam_sort_core] merging from 0 files and 16 in-memory blocks...
(MAPPING) 

: 1

# index

In [55]:
find $ref_dir -type f -name "*.sort.bam"|parallel -j 1 "samtools index -@$threads {}"

(MAPPING) 

: 1

# calculate depth

In [56]:
find $ref_dir -type f -name "*.sort.bam"

test/Acinetobacter_baumannii.fna.gz.mmi.sam.sort.bam
(MAPPING) 

: 1

In [59]:
samtools


Program: samtools (Tools for alignments in the SAM format)
Version: 1.7 (using htslib 1.7)

Usage:   samtools <command> [options]

Commands:
  -- Indexing
     dict           create a sequence dictionary file
     faidx          index/extract FASTA
     index          index alignment

  -- Editing
     calmd          recalculate MD/NM tags and '=' bases
     fixmate        fix mate information
     reheader       replace BAM header
     targetcut      cut fosmid regions (for fosmid pool only)
     addreplacerg   adds or replaces RG tags
     markdup        mark duplicates

  -- File operations
     collate        shuffle and group alignments by name
     cat            concatenate BAMs
     merge          merge sorted alignments
     mpileup        multi-way pileup
     sort           sort alignment file
     split          splits a file by read group
     quickcheck     quickly check if SAM/BAM/CRAM file appears intact
     fastq          converts a BAM to a FASTQ
     fasta         

: 1

In [60]:
find $ref_dir -type f -name "*.sort.bam"|parallel "samtools depth -aa {} > {}.depth"

(MAPPING) 

: 1

In [61]:
python script_depth.py $ref_dir

(MAPPING) 

: 1

# delete the temp files

In [ ]:
find $